# 向量数据库 安装与应用 Qdrant

In [ ]:
# 基础配置
from langchain_community.document_loaders import (
    TextLoader
)

# 初始化加载器（指定txt文件路径）
txt_loader = TextLoader("./file/book.txt")  # 替换为你的txt文件路径
txt_docs = txt_loader.load()

all_docs = txt_docs


from langchain_text_splitters import RecursiveCharacterTextSplitter
# 初始化分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Chunk最大长度（按字符数，或按token需配合tokenizer）
    chunk_overlap=200,  # 相邻Chunk重叠部分（保留上下文关联）
    separators=["\n\n", "\n", "。", "，", " "]  # 中文场景可添加中文分隔符
)


# 分割文档（输入为LangChain的Document列表）
split_docs = text_splitter.split_documents(all_docs)

## 三、嵌入模型加载与向量生成
### 1. 依赖安装

In [1]:
# Qdrant客户端
#!pip install qdrant-client

# 文本嵌入模型（Sentence-BERT）
#!pip install sentence-transformers

# Qwen-VL-Embedding依赖（需PyTorch）
#!pip install transformers torch modelscope  # modelscope用于加载阿里模型

### 纯文本嵌入 Sentence-BERT

In [3]:
from sentence_transformers import SentenceTransformer

# 加载轻量模型（384维向量）
sentence_bert = SentenceTransformer("all-MiniLM-L6-v2")

def text_to_vector_sentence_bert(text: str) -> list[float]:
    """将文本转为向量（Sentence-BERT）"""
    return sentence_bert.encode(text).tolist()

# 测试：生成"向量数据库"的嵌入
vec = text_to_vector_sentence_bert("向量数据库用于存储和搜索高维向量")
print(f"向量维度：{len(vec)}")  # 输出384
vec

向量维度：384


[0.0022243806160986423,
 0.11513707041740417,
 0.04315420240163803,
 0.04705599322915077,
 -0.033167723566293716,
 0.049800895154476166,
 0.09667057543992996,
 0.022579720243811607,
 -0.02519882284104824,
 -0.030926376581192017,
 0.07960636913776398,
 -0.09037758409976959,
 0.07036513090133667,
 -0.0975218415260315,
 -0.0450732558965683,
 -0.02352513000369072,
 0.04899483546614647,
 0.04916069284081459,
 -0.09331606328487396,
 -0.0006861919537186623,
 -0.01069854386150837,
 -0.039095401763916016,
 0.02838360145688057,
 0.05197634547948837,
 -0.1013290286064148,
 0.0642004907131195,
 0.004353360738605261,
 0.0007766913622617722,
 -0.0033870988991111517,
 -0.06303884834051132,
 -0.07262835651636124,
 -0.02219708077609539,
 -0.015869565308094025,
 0.015617793425917625,
 -0.0254006739705801,
 0.0057452707551419735,
 -0.05614722520112991,
 0.020169828087091446,
 0.024847710505127907,
 0.05887415260076523,
 0.005150679033249617,
 -0.040928833186626434,
 0.06004410982131958,
 0.00909868255257

###  Qwen-VL-Embedding（多模态嵌入）

In [ ]:
from transformers import AutoModel, AutoTokenizer

# 加载Qwen-VL-Embedding模型（768维向量）
model = AutoModel.from_pretrained(
    "qwen/Qwen-VL-Embedding",
    trust_remote_code=True,
    device_map="auto"  # 自动分配设备（CPU/GPU）
)
tokenizer = AutoTokenizer.from_pretrained(
    "qwen/Qwen-VL-Embedding",
    trust_remote_code=True
)

def text_to_vector_qwen(text: str) -> list[float]:
    """将文本转为向量（Qwen-VL-Embedding）"""
    inputs = tokenizer(text=text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        embeddings = model(**inputs, task_type="text_embedding").last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy().flatten().tolist()

# 测试：生成"多模态向量"的嵌入
vec = text_to_vector_qwen("多模态向量支持文本和图像的统一表示")
print(f"向量维度：{len(vec)}")  # 输出768

## 四、向量入库：分割文本→生成向量→存入Qdrant
假设已通过前文的文档加载和分割步骤，得到split_docs（LangChain的Document列表，每个含page_content和metadata），下面将其存入Qdrant。

### 1. 连接Qdrant客户端

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# 连接本地Qdrant服务
client = QdrantClient(host="localhost", port=6333)

### 2. 创建集合（Collection）
集合是Qdrant中存储向量的容器，需指定向量维度和距离度量方式（如余弦距离）：

In [6]:
# 选择嵌入模型（根据需求切换）
USE_MODEL = "sentence_bert"  # 或"qwen_vl"

# 定义向量参数
if USE_MODEL == "sentence_bert":
    vector_size = 384  # Sentence-BERT维度
else:
    vector_size = 768  # Qwen-VL-Embedding维度

# 创建集合（若已存在则先删除）
collection_name = "document_embeddings"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance.COSINE  # 文本相似性常用余弦距离
    )
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15480\2577244376.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

### 3. 批量插入向量与元数据
将分割后的文档转为向量，并关联元数据（如文本内容、来源文件、页码）存入集合：

In [2]:
#!pip install langchain_community

In [11]:
import uuid

# 准备插入数据（每个文档对应一个点（Point））
points = []
for doc in split_docs:
    # 生成向量
    if USE_MODEL == "sentence_bert":
        vector = text_to_vector_sentence_bert(doc.page_content)
    else:
        vector = text_to_vector_qwen(doc.page_content)
    
    # 构建点（含唯一ID、向量、元数据）
    points.append({
        "id": str(uuid.uuid4()),  # 唯一标识
        "vector": vector,
        "payload": {
            "text": doc.page_content,  # 原始文本
            "source": doc.metadata.get("source", "unknown"),  # 来源文件
            "page": doc.metadata.get("page", -1)  # 页码（PDF文档）
        }
    })

# 批量插入（批量操作效率更高）
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"成功插入{len(points)}个向量到集合{collection_name}")

成功插入29个向量到集合document_embeddings


## 五、相似性查询：从向量库检索相关文本
存入向量后，可通过查询文本生成向量，在Qdrant中搜索最相似的结果：

In [16]:
def search_similar(query: str, top_k: int = 3) -> list:
    """搜索与查询文本相似的文档"""
    # 生成查询向量
    if USE_MODEL == "sentence_bert":
        query_vec = text_to_vector_sentence_bert(query)
    else:
        query_vec = text_to_vector_qwen(query)
    
    # 搜索相似向量
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vec,
        limit=top_k  # 返回前3个最相似结果
    )
    
    # 整理结果（提取元数据和相似度分数）
    return [
        {
            "text": hit.payload["text"],
            "source": hit.payload["source"],
            "page": hit.payload["page"],
            "score": hit.score  # 相似度分数（0~1，越高越相似）
        }
        for hit in results
    ]

# 测试查询
query = "负荷开关"
similar_docs = search_similar(query)
for i, doc in enumerate(similar_docs, 1):
    print(f"\n第{i}个相似文档（分数：{doc['score']:.2f}）：")
    print(f"来源：{doc['source']}（第{doc['page']}页）")
    print(f"内容：{doc['text'][:200]}...")  # 打印前200字


第1个相似文档（分数：0.29）：
来源：./file/book.txt（第-1页）
内容：附录D（资料性附录）操作项目主、子项填写示例

顺序	操作项目	操作√
1	…	
2	在×××保护屏：	
	1）检查保护电源指示正确。	
	2）检查保护跳闸出口1LP1连接片在投入位置。	
	3）检查重合闸出口1LP2连接片在投入位置。	
	4）…	
3	…...

第2个相似文档（分数：0.22）：
来源：./file/book.txt（第-1页）
内容：在××kV××线××配电变压器××kV侧装设××kV（××号）接地线一组。
拆除××kV××线××配电变压器××kV侧××kV（××号）接地线一组。
（2）验明××kV××线××隔离开关（负荷开关）线路侧三相确无电压。
在××kV××线××隔离开关（负荷开关）线路侧装设××kV（××号）接地线一组。
拆除××kV××线××隔离开关（负荷开关）线路侧××kV（××号）接地线一组。
（3）验明××kV...

第3个相似文档（分数：0.20）：
来源：./file/book.txt（第-1页）
内容：l）投入、退出相关的二次连接片；投入出口、联跳连接片前，测量连接片对地电压；
m）投入、退出断路器、隔离开关等设备的操作电源（包括控制电源、电机电源或合闸电源）；
n）在具体位置检验确无电压（合接地开关、装设接地线前）；
o）对于无人值班变电站的操作，应根据操作任务核对相关设备的运行方式；
p）装设或拆除绝缘挡板或绝缘罩；
q）核对现场设备的运行状态；
r）线路检修状态转换时，悬挂和取下标示牌；
...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15480\2554484837.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


## 六、优化与注意事项
1. 索引优化：Qdrant默认使用HNSW索引，可通过调整hnsw_config提升搜索效率（如m=16、ef_construct=200）：

In [ ]:
from qdrant_client.models import HnswConfig

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    hnsw_config=HnswConfig(m=16, ef_construct=200)  # m：每个节点的邻居数；ef_construct：构建时的探索深度
)

2. 批量处理：插入大量向量时，建议分批次（如每批1000个）调用upsert，避免内存占用过高。
3. 模型选择：  
  - 纯文本场景优先用Sentence-BERT（速度快、维度低，节省存储）；  
  - 含图片的文档用Qwen-VL-Embedding（需额外处理图像嵌入，如用model.encode_image()）。
4. 元数据过滤：查询时可通过filter参数按来源、页码等元数据筛选结果：

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

results = client.search(
    collection_name=collection_name,
    query_vector=query_vec,
    limit=3,
    filter=Filter(
        must=[FieldCondition(key="source", match=MatchValue(value="文档.pdf"))]  # 只搜索来自文档.pdf的结果
    )
)